In [14]:
import h5py
import numpy as np
import os
import sys
import pandas as pd
import scipy.io as scio
import torch
import matplotlib.pyplot as plt
from src.torchHHT import hht
from src.hht.hht import signal_window

In [15]:
data_dir = '../../data/input/CWRU/raw_data/'

frame = pd.read_table('annotations.txt')

dim = 2048

train_fraction = 0.8

signal_train = []
label_train = []
signal_test = []
label_test = []

count = 0


In [33]:
for index in range(len(frame)):
    mat_name = os.path.join(data_dir, frame['file_name'][index])
    raw_data = scio.loadmat(mat_name)
    for key, data in raw_data.items():
        if key[5:7] == 'DE':
            signal = data.squeeze()
            temp_signal = signal
            # signal = signal_window(signal, 0.3)
            fs = signal.shape[0]
            if torch.cuda.is_available():
                signal = torch.Tensor(signal)
                signal = signal.cuda()
                pass
            imfs, imfs_env, imfs_freq = hht.hilbert_huang(signal, fs=fs, num_imf=10)
            imfs = imfs.cpu().numpy()
            imfs = np.sum(imfs[1:4], axis=0)
            # print(imfs)
            
            sample_num = imfs.shape[0] // dim
            
            train_num = int(sample_num * train_fraction)
            test_num = sample_num - train_num
            
            signal = imfs[0: dim * sample_num]
            
            signals = np.array(np.split(signal, sample_num))
            
            signal_train.append(signals[0: train_num])
            signal_test.append(signals[train_num: sample_num])
            
            label_train.append(index * np.ones(train_num))
            label_test.append(index * np.ones(test_num))
            

            # plt.figure(figsize=(20, 10))
            # plt.plot(temp_signal, label='origin')
            # plt.plot(imfs, label='imf')
            # plt.legend()
                
        pass
    pass



In [42]:
len(signal_train)

306

In [53]:
signals_tr_np = np.concatenate(signal_train)
labels_tr_np = np.concatenate(np.array(label_train)).astype('uint8')
signals_tt_np = np.concatenate(signal_test)
labels_tt_np = np.concatenate(np.array(label_test)).astype('uint8')
print(signals_tr_np.shape, labels_tr_np.shape, signals_tt_np.shape, labels_tt_np.shape)

(16209, 2048) (16209,) (4140, 2048) (4140,)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1464\825072348.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels_tr_np = np.concatenate(np.array(label_train)).astype('uint8')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1464\825072348.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels_tt_np = np.concatenate(np.array(label_test)).astype('uint8')


In [33]:
signals_tr_np

array([[ 0.04704444,  0.08269608,  0.09427672, ...,  0.0402003 ,
         0.02143299, -0.02502519],
       [-0.04735624, -0.04193234, -0.00679454, ...,  0.16723057,
         0.17658587,  0.15072723],
       [ 0.0658972 , -0.03534985, -0.09689139, ...,  0.08966754,
         0.10510067,  0.0953364 ],
       ...,
       [-0.07989481, -0.07793628, -0.04873594, ..., -0.01656776,
        -0.01328027, -0.00953359],
       [-0.00777308, -0.02118734, -0.04230409, ..., -0.09707723,
        -0.0752073 , -0.02518051],
       [ 0.03697373,  0.09343565,  0.12315738, ..., -0.03135856,
        -0.00453479, -0.00152512]])

In [34]:
labels_tr_np

array([  0,   0,   0, ..., 100, 100, 100], dtype=uint8)

In [74]:
f = h5py.File('DE1.h5', 'w')
f.create_dataset('X_train', data=signals_tr_np)
f.create_dataset('y_train',data=labels_tr_np)
f.create_dataset('X_test', data=signals_tt_np)
f.create_dataset('y_test',data=labels_tt_np)
f.close()

# CWRU CNN

In [1]:
from CWRUCNN import CWRUCNN

In [37]:
model = CWRUCNN()

In [39]:
print(model)

CWRUCNN(
  (layer): Sequential(
    (0): Conv1d(1, 81, kernel_size=(55,), stride=(1,))
    (1): BatchNorm1d(81, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(81, 27, kernel_size=(55,), stride=(1,))
    (5): BatchNorm1d(81, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Conv1d(81, 81, kernel_size=(55,), stride=(1,))
    (9): BatchNorm1d(81, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool1d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)
    (12): Conv1d(81, 27, kernel_size=(55,), stride=(1,))
    (13): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.5, inplace=False)
    (16): Conv1d(27, 27, kernel_size=(55,), stride=(1,))
    (17): Ba

In [9]:
from src.CWRU.config import Config
from src.CWRU.CWRUdata import CWRUdata
from src.CWRU.utils import create_dataset, create_model
from torch.utils.data import DataLoader
from utils import check_accuracy
import torch
from tensorboardX import SummaryWriter
import copy
import time
import pandas as pd
from torchsummary import summary


In [3]:
opt = Config()

In [55]:
train_dataset = create_dataset(opt.train_dir, 'train')
train_dataloader = DataLoader(train_dataset, batch_size=opt.batch_size, shuffle=True)

dataset [DE1.h5] was created


In [56]:
val_dataset = create_dataset(opt.val_dir, 'val')
val_dataloader = DataLoader(val_dataset, batch_size=opt.batch_size, shuffle=True)

dataset [DE1.h5] was created


In [50]:
print('#training num = %d' % len(train_dataset))
print('#val num = %d' % len(val_dataset))

#training num = 16209
#val num = 16209


In [28]:
model = create_model(opt.model, opt.model_param)
writer = SummaryWriter()

model [plain_cnn] was created


In [29]:
print(model)

CWRUCNN(
  (layer): Sequential(
    (0): Conv1d(1, 27, kernel_size=(55,), stride=(1,), padding=(27,))
    (1): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(27, 27, kernel_size=(55,), stride=(1,), padding=(27,))
    (5): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Conv1d(27, 27, kernel_size=(55,), stride=(1,), padding=(27,))
    (9): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool1d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)
    (12): Conv1d(27, 27, kernel_size=(55,), stride=(1,), padding=(27,))
    (13): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.5, inplace=False)
    (16):

## 模型训练

In [24]:
total_steps = 0
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=opt.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, opt.lr_decay_iters, opt.lr_decay)
loss_fn = torch.nn.CrossEntropyLoss()

In [64]:
use_cuda = torch.cuda.is_available()

if use_cuda:
    print('CUDA is available')

device = torch.device(opt.device if use_cuda else "cpu")
model.to(device)

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

CUDA is available


In [73]:
for epoch in range(opt.epochs):
    t0 = time.time()
    print('Starting epoch %d / %d' % (epoch + 1, opt.epochs))
    scheduler.step()
    model.train()
    
    for t, (x, y) in enumerate(train_dataloader):
        # print(x)
        # print(x.size())
        
        x.resize(x.size()[0], x.size()[1])
        x = x.unsqueeze(1)
        # print(x)
        # print(x.size())
        
        
        x, y = x.float(), y.long()
        x, y = x.to(device), y.to(device)
        score = model(x)
        loss = loss_fn(score, y)
        writer.add_scalar('loss', loss.item())
        
        if (t + 1) % opt.print_every:
            print('t = %d, loss = %.4f' % (t + 1, loss.item()))
            pass
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pass
    
    train_acc, _ = check_accuracy(model, train_dataloader, device)
    val_acc, _ = check_accuracy(model, val_dataloader, device)
    
    # 添加acc 与 权重到tensorboard
    
    writer.add_scalar('train_acc', train_acc, epoch)
    writer.add_scalar('val_acc', val_acc, epoch)
    
    for name, param in model.named_parameters():
        writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)
    # save the best model
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        
    t1 = time.time()
    
    print(t1 - t0)
    
    pass

# 输出结果
print('kernel num1: {}'.format(opt.model_param['kernel_num1']))
print('kernel num2: {}'.format(opt.model_param['kernel_num2']))
print('Best val Acc: {:4f}'.format(best_acc))

Starting epoch 1 / 50
t = 1, loss = 4.6653
t = 2, loss = 4.6962
t = 3, loss = 4.6044
t = 4, loss = 4.8122


E:\ProgramData\Anaconda3\lib\site-packages\torch\_tensor.py:836: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


t = 5, loss = 4.6486
t = 6, loss = 4.6666
t = 7, loss = 4.6530
t = 8, loss = 4.7134
t = 9, loss = 4.7190
t = 10, loss = 4.6011
t = 11, loss = 4.7414
t = 12, loss = 4.7123
t = 13, loss = 4.6693
t = 14, loss = 4.6730
t = 15, loss = 4.7564
t = 16, loss = 4.6177
t = 17, loss = 4.7147
t = 18, loss = 4.6931
t = 19, loss = 4.7698
t = 20, loss = 4.7320
t = 21, loss = 4.7744
t = 22, loss = 4.7408
t = 23, loss = 4.6876
t = 24, loss = 4.6899
t = 25, loss = 4.7890
t = 26, loss = 4.6783
t = 27, loss = 4.8243
t = 28, loss = 4.6587
t = 29, loss = 4.6620
t = 30, loss = 4.8229
t = 31, loss = 4.6158
t = 32, loss = 4.8040
t = 33, loss = 4.8359
t = 34, loss = 4.8384
t = 35, loss = 4.5708
t = 36, loss = 4.8364
t = 37, loss = 4.7501
t = 38, loss = 4.6740
t = 39, loss = 4.6495
t = 40, loss = 4.7421
t = 41, loss = 4.7724
t = 42, loss = 4.6972
t = 43, loss = 4.7933
t = 44, loss = 4.7829
t = 45, loss = 4.8089
t = 46, loss = 4.7248
t = 47, loss = 4.7626
t = 48, loss = 4.6209
t = 49, loss = 4.6438
t = 50, loss = 